# Fallabella's Sneakers Analysis

The objective is to gather information about the characteristics of sneakers, including their prices, discounts, and original costs, in order to examine potential emerging correlations between discounts, pricing, and specific attributes. This analysis aims to validate if it is possible to:

- Predict product pricing
- Predict/determine discount proportions (for instances when people have Falabella's credit card and when they do not)
- Descriptive analysis of the products

### 1. Understanding web page structure



In [124]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?facetSelected=true&f.derived.variant.sellerId=FALABELLA")

In [ ]:
# create a function to get the brands of the products
def getproduct_brand():
    # reload the page
    driver.refresh()
    brands = driver.find_elements(By.CLASS_NAME, "title-rebrand")
    for brand in brands:
        print(brand.text)

print(getproduct_brand())

In [ ]:
# create a function to get the names of the products
def getproduct_name():
    names = driver.find_elements(By.CLASS_NAME, "pod-subTitle")
    for name in names:
        print(name.text)
        

print(getproduct_name())


In [ ]:
# getting price 

def getproduct_price():
    prices = driver.find_elements(By.CLASS_NAME, "jsx-2112733514") 
    counter = 0
    
    for price in prices:
        print(price.text)
        # print product counter
        counter += 1
        print(f"product {counter}")
        # select li elements
        price_items = price.find_elements(By.TAG_NAME, "li")
        # iterate over the li elements
        
        for item in price_items:

            if item.get_attribute("data-cmr-price"):
                cmr_price = item.get_attribute("data-cmr-price")
                print("precio cmr:" + cmr_price)
            if item.get_attribute("data-event-price"):
                event_price = item.get_attribute("data-event-price")
                print("precio descuento:" + event_price)
            if item.get_attribute("data-normal-price"):
                full_price = item.get_attribute("data-normal-price")
                print("precio full:" + full_price)       
        if None in prices:
            continue
        
        counter += 1
    
                    

print(getproduct_price())

### 1.1 looking at sneakers details


In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

def get_product_details_url():
    # get the names of the products
    names = driver.find_elements(By.CLASS_NAME, "pod-subTitle")
    element = names[1]
    # create an action chain object to click on the element and expand the product details
    actions = ActionChains(driver)
    actions.move_to_element(element).click().perform()
    # wait to the element to be clickable
    driver.implicitly_wait(10)
    buttons = driver.find_elements(By.CLASS_NAME, "mkp-swatchButton-collapseButton")
    for button in buttons:
        if button.text == "ver más":
             button.click()
            
    
    table = driver.find_element(By.CLASS_NAME, "specification-table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        categories = row.find_element(By.TAG_NAME, "td")
        print(categories.text) # this will be columns of the data frame 
        values = row.find_elements(By.TAG_NAME, "td")
        print(values[1].text) # this will be the values of the data frame according to the columns
           
    # then, we have to go back to the previous page to get the next product details
    
get_product_details_url()

## 2. Implementation

### Guided Path:

1. **pages**: Number of pages of the sneaker section.
2. **pages_links**: All the hrefs of all pages.
3. **get_href()**: Retrieves all direct products of each page.
4. **get_all_links()**: Appends all the hrefs of all pages of the sneaker section.
5. **clean_link()**: Formats the links to avoid errors.
6. **pricing()**: Retrieves CMR, normal, and event prices of each product.
7. **extract_details()**: Extracts all the attributes of each pair of sneakers.
8. **create_df()**: Utilizes all the logging in the first section, pricing, and links extraction function in order to gather all product information.


In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA")

In [ ]:
pages = driver.find_elements(By.CLASS_NAME, "jsx-1389196899")
print(len(pages))

32


In [ ]:
pages_links = []
pages_links.append("https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA")
for i in range(2, len(pages) + 1):
    link = f"https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page={i}"
    pages_links.append(link)
print(pages_links)

['https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=2', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=3', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=4', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=5', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=6', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.sellerId=FALABELLA&facetSelected=true&page=7', 'https://www.falabella.com.co/falabella-co/category/cat6360942/Tenis?f.derived.variant.

In [ ]:
# get href of the atribute a in the actual page
def get_href():
    detail_links = []
    hrefs = driver.find_elements(By.CLASS_NAME, "pod-2_GRID")
    for href in hrefs:
        detail_links.append(href.get_attribute("href"))
    return detail_links

detail_links = get_href()


In [ ]:
def get_all_links():
    all_links = []
    for link in pages_links:
        print(link)
        driver.get(link)
        boxes = driver.find_elements(By.CLASS_NAME, "pod-link")
        print(len(boxes))
        for box in boxes:
            box.get_attribute("href")
            print(box.get_attribute("href"))
            all_links.append(box.get_attribute("href"))
        
        print(all_links)
    return all_links
            

all_links = get_all_links()
# make all links a csv file
import pandas as pd

df = pd.DataFrame(all_links)
df.to_csv("all_links.txt", index=False)

In [91]:
# remove all " from the links
def clean_links():
    clean_links = []
    for link in links:
        clean_link = link.replace('"', "")
        clean_links.append(clean_link)
    return clean_links

clean_links = clean_links()
print(clean_links)

# save the clean links in a txt file
def save_clean_links():
    with open('clean_links.txt', 'w') as file:
        for link in clean_links:
            file.write(link + "\n")

save_clean_links()

['https://www.falabella.com.co/falabella-co/product/72889166/Tenis-Adidas-para-Mujer-Moda-Court-Platform/72889168']


In [ ]:
# read clean links from the txt file and store them in a list
def read_clean_links():
    links = []
    with open('clean_links.txt', 'r') as file:
        for link in file:
            links.append(link)
    return links

links = read_clean_links()

In [81]:
def pricing(driver):
    cmr_price = event_price = full_price = None  # Initialize the variables

    # Attempt to fetch each type of price, handle exceptions if elements are not found
    try:
        cmr_elements = driver.find_elements(By.CLASS_NAME, "prices-0")
        if cmr_elements:
            cmr_price = cmr_elements[0].get_attribute("data-cmr-price")
    except Exception:
        pass  # If not found, simply continue

    try:
        event_elements = driver.find_elements(By.CLASS_NAME, "prices-1")
        if event_elements:
            event_price = event_elements[0].get_attribute("data-event-price")
    except Exception:
        pass  # If not found, simply continue

    try:
        full_elements = driver.find_elements(By.CLASS_NAME, "prices-2")
        if full_elements:
            full_price = full_elements[0].get_attribute("data-normal-price")
    except Exception:
        pass  # If not found, simply continue

    # Fallback strategies for other price configurations
    try:
        if not full_price:
            full_elements = driver.find_elements(By.CLASS_NAME, "prices-0")
            if full_elements:
                full_price = full_elements[0].get_attribute("data-internet-price")
    except Exception:
        pass  # If not found, simply continue

    return cmr_price, event_price, full_price

## Data Extraction

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def extract_details(driver):
    # Click en los botones "ver más" si están presentes
    buttons = driver.find_elements(By.CLASS_NAME, "mkp-swatchButton-collapseButton")
    for button in buttons:
        if button.text == "ver más":
            button.click()
            print("button clicked")
    
    # Buscar la tabla de especificaciones y procesar cada fila
    table = driver.find_element(By.CLASS_NAME, "specification-table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    attributes = {}
    for row in rows:
        process_row(row, attributes)
    return attributes

def process_row(row, attributes):
    try:
        values = row.find_elements(By.TAG_NAME, "td")
        if len(values) < 2:
            return

        attribute_name = values[0].text.strip().lower()  # Normalize the attribute name to lower case
        attribute_value = values[1].text.strip() if values[1].text else None

        # Map attribute names to their corresponding variable names explicitly
        attribute_mapping = {
            "marca": "brand",
            "género": "gender",
            "tipo": "type",
            "horma": "last_type",
            "material": "material",
            "material del forro": "lining_material",
            "material de la suela": "sole_material",
            "material del interior": "interior_material",
            "temporada": "season",
            "hecho en": "made_in",
            "disciplina": "discipline",
            "material de la plantilla": "insole_material"
        }

        if attribute_name in attribute_mapping:
            attributes[attribute_mapping[attribute_name]] = attribute_value
            print(f"{attribute_mapping[attribute_name]}: {attribute_value}")

    except Exception as e:
        print(f"An exception occurred while processing row: {e}")

def create_df(links):
    driver = webdriver.Chrome()

    all_data = pd.DataFrame()
    

    for link in links:
        print("Scraping:", link)
        driver.get(link)
        try:
            product_name = driver.find_element(By.CLASS_NAME, "product-name").text
            product_rating = driver.find_element(By.CLASS_NAME, "_text_17o93_5").text
            product_code = driver.find_element(By.CLASS_NAME, "jsx-3410277752").text
            
            prices = pricing(driver)
            attributes = extract_details(driver)
            
            # Include prices data in attributes dictionary
            attributes.update({
                'Name': [product_name],
                'Rating': [product_rating],
                'Code': [product_code],
                'CMR Price': prices[0],
                'Event Price':prices[1],
                'Full Price': prices[2]
            })

            temp_df = pd.DataFrame([attributes])
            all_data = pd.concat([all_data, temp_df], ignore_index=True)

        except Exception as e:
            print(f"An error occurred while scraping {link}: {e}")

    driver.quit()
    return all_data

# Call the function and print the DataFrame

first_df = create_df(links=read_clean_links())
first_df


In [122]:
# export the dataframe to a csv file
first_df.to_csv("first_df_final.csv", index=False)

# 3. Data Cleaning

The *Rating* , *Name* and *Code* columns may have extraneous information enclosed in square brackets.

In [145]:
df = pd.read_csv("first_df_final.csv")

In [146]:
df

,brand,type,gender,last_type,material,lining_material,sole_material,season,made_in,insole_material,Name,Rating,Code,CMR Price,Event Price,Full Price,discipline,interior_material
0,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,['Tenis Skechers para Mujer Moda Slip Ins Ultr...,['(0)'],['Código: 72778443'],233.89,272.89,389.900,NaN,NaN
1,Under Armour,Tenis deportivos,Mujer,Normal,Tela,Textil,Caucho,Toda temporada,Indonesia,NaN,['Tenis Under Armour Mujer Running Surge 3'],['4.9 (17)'],['Código: 42790553'],216.89,247.89,309.900,Running,NaN
2,Reebok,Tenis,Mujer,Normal,Sintético,Textil,Goma,Toda temporada,Vietnam,Textil,['Tenis Reebok para Mujer Cross Training Nanof...,['4 (2)'],['Código: 72804665'],272.99,311.99,389.990,Cross training,NaN
3,Nike,Tenis,Hombre,Normal,Tela,Poliéster,Goma,Toda temporada,Indonesia,Poliéster,['Tenis Nike para Hombre Running Revolution7'],['(0)'],['Código: 72875577'],NaN,NaN,344.990,Running,NaN
4,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,['Tenis Skechers para Mujer Moda Go Walk Air 2...,['(0)'],['Código: 72778360'],227.89,265.89,379.900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,New Balance,Tenis moda,Hombre,Normal,Sintético,Textil,Caucho,Toda temporada,Vietnam,Textil,['Tenis New Balance para Hombre Moda 550'],['5 (3)'],['Código: 72735458'],NaN,NaN,859.990,NaN,NaN
1526,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,['Tenis Skechers para Mujer Moda Summits Perfe...,['(0)'],['Código: 72890296'],179.99,199.99,279.900,Lifestyle,NaN
1527,Adidas,Tenis deportivos,Mujer,Normal,Tela,Textil,Caucho,Toda temporada,Vietnam,Textil,['Tenis Adidas Mujer Cross training Rapidmove ...,['4.6 (41)'],['Código: 72712203'],454.99,519.99,649.950,Cross training,NaN
1528,Adidas,Tenis deportivos,Hombre,Normal,Tela,Textil,Caucho,Toda temporada,China,Textil,['Tenis Adidas para Hombre Running Adizero SL'],['4.8 (779)'],['Código: 72889124'],NaN,NaN,699.990,Running,NaN


In [147]:
# deleting The *Rating* , *Name* and *Code* [] characters from the dataframe
df['Rating'] = df['Rating'].str[2:-2]
df['Name'] = df['Name'].str[2:-2]
df['Code'] = df['Code'].str[2:-2]


In [149]:
import numpy as np
# if rating is (0) asign a value of null to it
df['Rating'] = df['Rating'].apply(lambda x: None if x == '(0)' else x)  # Replace (0) with None

# and if it has 4.9 (17) strucutre, asign 4.9 to it 
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0] if x else None)  # Extract the rating value from the string

# replace all NaN or "NaN" with None
df.replace({np.nan: None, 'NaN': None}, inplace=True)


In [150]:
df

,brand,type,gender,last_type,material,lining_material,sole_material,season,made_in,insole_material,Name,Rating,Code,CMR Price,Event Price,Full Price,discipline,interior_material
0,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,Tenis Skechers para Mujer Moda Slip Ins Ultra ...,None,Código: 72778443,233.89,272.89,389.900,None,None
1,Under Armour,Tenis deportivos,Mujer,Normal,Tela,Textil,Caucho,Toda temporada,Indonesia,None,Tenis Under Armour Mujer Running Surge 3,4.9,Código: 42790553,216.89,247.89,309.900,Running,None
2,Reebok,Tenis,Mujer,Normal,Sintético,Textil,Goma,Toda temporada,Vietnam,Textil,Tenis Reebok para Mujer Cross Training Nanofle...,4,Código: 72804665,272.99,311.99,389.990,Cross training,None
3,Nike,Tenis,Hombre,Normal,Tela,Poliéster,Goma,Toda temporada,Indonesia,Poliéster,Tenis Nike para Hombre Running Revolution7,None,Código: 72875577,None,None,344.990,Running,None
4,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,Tenis Skechers para Mujer Moda Go Walk Air 2.0...,None,Código: 72778360,227.89,265.89,379.900,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,New Balance,Tenis moda,Hombre,Normal,Sintético,Textil,Caucho,Toda temporada,Vietnam,Textil,Tenis New Balance para Hombre Moda 550,5,Código: 72735458,None,None,859.990,None,None
1526,Skechers,Tenis moda,Mujer,Normal,Sintético,Poliéster,Goma,Toda temporada,China,Poliéster,Tenis Skechers para Mujer Moda Summits Perfect...,None,Código: 72890296,179.99,199.99,279.900,Lifestyle,None
1527,Adidas,Tenis deportivos,Mujer,Normal,Tela,Textil,Caucho,Toda temporada,Vietnam,Textil,Tenis Adidas Mujer Cross training Rapidmove Ad...,4.6,Código: 72712203,454.99,519.99,649.950,Cross training,None
1528,Adidas,Tenis deportivos,Hombre,Normal,Tela,Textil,Caucho,Toda temporada,China,Textil,Tenis Adidas para Hombre Running Adizero SL,4.8,Código: 72889124,None,None,699.990,Running,None


In [152]:
clean_df = df
# save clean_df to a csv file
clean_df.to_csv("clean_df.csv", index=False)

# Describing Data

In [159]:
clean_df = pd.read_csv("clean_df.csv")

In [160]:
# counting nulls 
round(clean_df.isnull().sum()/len(clean_df) * 100,2)



brand                 0.00
type                  0.00
gender                0.00
last_type             1.57
material              0.00
lining_material      21.63
sole_material         0.07
season                0.00
made_in               0.07
insole_material      19.54
Name                  0.00
Rating               22.09
Code                  0.00
CMR Price            64.71
Event Price          69.35
Full Price           10.65
discipline           50.85
interior_material    78.43
dtype: float64

In [161]:
# data types of the columns
clean_df.dtypes

brand                 object
type                  object
gender                object
last_type             object
material              object
lining_material       object
sole_material         object
season                object
made_in               object
insole_material       object
Name                  object
Rating               float64
Code                  object
CMR Price            float64
Event Price          float64
Full Price            object
discipline            object
interior_material     object
dtype: object

In [163]:
# Remove dots from 'Full Price' column
clean_df['Full Price'] = clean_df['Full Price'].str.replace('.', '')

# Convert 'Full Price' column to float
clean_df['Full Price'] = clean_df['Full Price'].astype(float)


columns = ['brand', 'type', 'gender', 'last_type', 'material', 'lining_material', 'sole_material', 'interior_material', 'season', 'made_in', 'discipline', 'insole_material']
for column in columns:
    clean_df[column] = clean_df[column].astype(str)


In [180]:
categories = pd.DataFrame()
yes = {'column': [], 'value': []}  # Initialize the 'yes' dictionary
    
for column in columns:
    unique_values = clean_df[column].unique()
    for value in unique_values:
        yes['column'].append(column)
        yes['value'].append(value)

categories = pd.DataFrame(yes)  # Convierte el diccionario a un DataFrame

categories


,column,value
0,brand,Skechers
1,brand,Under Armour
2,brand,Reebok
3,brand,Nike
4,brand,Diadora
...,...,...
143,insole_material,Malla (Nylon)
144,insole_material,Caucho
145,insole_material,Algodón
146,insole_material,Cuero
